In [1]:
import sqlite3
import json
import cv2
import numpy as np
import base64
from keras.models import load_model

model1 = load_model('stage1-epoch-150.h5')
model2 = load_model('stage2-epoch-150.h5')

Using TensorFlow backend.
/Users/zzj/Documents/AR/py27/lib/python2.7/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
def MakeLandmarkImage(landmarks):
    res = np.zeros((256, 256), np.uint8)
    for cor in landmarks:
        x = int(cor[1]*256/500)
        y = int(cor[0]*256/500)
        res[x,y] = 255
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
    res = cv2.dilate(res,kernel,iterations = 1)

    return res

def MakeSketchImage(input):
    res = cv2.split(input)[3]
    res = 255-res
    res = cv2.resize(res, (256, 256))

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    res = cv2.erode(res,kernel,iterations = 1)
    _, res = cv2.threshold(res,200,255,cv2.THRESH_BINARY)

    return res

## Two-stage

In [5]:
conn = sqlite3.connect('participants.db')
cur = conn.cursor()

cur.execute('SELECT uniqueid, datastring FROM turkdemo')
for row in cur:
#     print(row[0])
    try:
        container = json.loads(row[1])
        container = container['questiondata']
        if 'user_id' in container:
            id = container['user_id']
            data = container['all_data']
            time = data['time']
            filename = data['filename']
            landmark = data['landmark']
            sketch = data['sketch']
            
            encoded_image = sketch.split(",")[1]
            decoded_image = base64.b64decode(encoded_image)

            arr = np.asarray(bytearray(decoded_image), dtype=np.uint8)
            img = cv2.imdecode(arr, -1) # Load it as it is
            
            landmark_image = MakeLandmarkImage(landmark)
            sketch_image = MakeSketchImage(img)
        
            landmark_image = cv2.cvtColor(landmark_image, cv2.COLOR_GRAY2BGR)
            sketch_image = cv2.cvtColor(sketch_image, cv2.COLOR_GRAY2BGR)
            
            cv2.imwrite('./results/landmark/'+id+'.png', landmark_image)
            cv2.imwrite('./results/sketch/'+id+'.png', sketch_image)
            
            landmark_image = np.array(landmark_image)/127.5-1
            sketch_image = np.array(sketch_image)/127.5-1

            landmark_image = landmark_image[np.newaxis,...]
            sketch_image = sketch_image[np.newaxis,...]
            
            fake_sketch = model1.predict(landmark_image)
            sketch = np.concatenate([fake_sketch, sketch_image], axis=3)
            result = model2.predict(sketch)
            
            result = np.squeeze((result * 127.5 + 127.5).astype(np.uint8))
            
            cv2.imwrite('./results/image/'+id+'.png', result)
            
            source = cv2.imread('./show/'+filename, 1)
            blend = cv2.addWeighted(source,0.1,result,0.9,0)
                        
            cv2.imwrite('./results/blend/'+id+'.png', blend)
            
            print(id)
    except:
        pass
    

livelGKhh:livecAe8A
liveqiCpO:livekxnSI
liveZalZG:live3mmqv
liverKKXQ:liveBmTbT
liveRrGr5:liveNQ5pJ
liveTnIGw:liveDDuPv
liveWCitR:liveT90xQ
livetUVW6:live3JoJ8
liveirB7B:liveALieQ
liveMQzbL:live86ZrM
livehnnVT:live8nbIU
liveUwrB8:liveNZ5pU
livecGEHE:liveSkNnu
liveu8J6I:livexahWM
livegIjan:liveq34iy
liveU6dmX:livebGMPY
liveiVMty:livee906x
live1odip:live4dECl
liveaXNpX:liveetqUy
liveVe3AD:liveDe6mQ
live4o2ot:live1VkMU
live70nqI:livezjkam
live5chJb:livekwxeg
livesacqn:liveKVY8k
liveljo8R:livevgXNH
live2ZWHQ:live1Vwq7
liveFOhlh:liveXGHLz
liveK6dPs:livekq2IY
liveMLwoJ:liveUcZsS
liveawLoL:liveuNoce
liveBiXqb:liveVVVl7
liveqIV8z:liveRfvLA
liveiFkP1:liveiD8hw
liveMAA9l:livedCfIX
liveoUMSN:liveQkFdl
live4fwRy:livenu1tl
livewIsrT:liveUrjIt
livenqXfE:liveiNCVm
liveoiiSl:livemSkpP
livef3xCx:live9OxXe
live842lX:liveyfgt9
liveRZWx2:liveFdY1z
liveBCvy3:livez62ur
liveE5AaQ:liveJRkqM
liveMdFCj:live9Lz2r
live1R2sq:liveQce5E


## Pix2pix

In [6]:
pix2pix_model = load_model('pix2pix-epoch-149.h5')

In [7]:
conn = sqlite3.connect('participants.db')
cur = conn.cursor()

cur.execute('SELECT uniqueid, datastring FROM turkdemo')
for row in cur:
#     print(row[0])
    try:
        container = json.loads(row[1])
        container = container['questiondata']
        if 'user_id' in container:
            id = container['user_id']
            data = container['all_data']
            time = data['time']
            filename = data['filename']
            landmark = data['landmark']
            sketch = data['sketch']

            encoded_image = sketch.split(",")[1]
            decoded_image = base64.b64decode(encoded_image)

            arr = np.asarray(bytearray(decoded_image), dtype=np.uint8)
            img = cv2.imdecode(arr, -1) # Load it as it is

            sketch_image = MakeSketchImage(img)

            sketch_image = cv2.cvtColor(sketch_image, cv2.COLOR_GRAY2BGR)

            sketch_image = np.array(sketch_image)/127.5-1

            sketch_image = sketch_image[np.newaxis,...]

            pix2pix_result = pix2pix_model.predict(sketch_image)

            pix2pix_result = np.squeeze((pix2pix_result * 127.5 + 127.5).astype(np.uint8))

            cv2.imwrite('./results/pix2pix/'+id+'.png', pix2pix_result)

            source = cv2.imread('./show/'+filename, 1)

            pix2pix_blend = cv2.addWeighted(source,0.1,pix2pix_result,0.9,0)

            cv2.imwrite('./results/pix2pix_blend/'+id+'.png', pix2pix_blend)

            print(id)
    except:
        pass
    

livelGKhh:livecAe8A
liveqiCpO:livekxnSI
liveZalZG:live3mmqv
liverKKXQ:liveBmTbT
liveRrGr5:liveNQ5pJ
liveTnIGw:liveDDuPv
liveWCitR:liveT90xQ
livetUVW6:live3JoJ8
liveirB7B:liveALieQ
liveMQzbL:live86ZrM
livehnnVT:live8nbIU
liveUwrB8:liveNZ5pU
livecGEHE:liveSkNnu
liveu8J6I:livexahWM
livegIjan:liveq34iy
liveU6dmX:livebGMPY
liveiVMty:livee906x
live1odip:live4dECl
liveaXNpX:liveetqUy
liveVe3AD:liveDe6mQ
live4o2ot:live1VkMU
live70nqI:livezjkam
live5chJb:livekwxeg
livesacqn:liveKVY8k
liveljo8R:livevgXNH
live2ZWHQ:live1Vwq7
liveFOhlh:liveXGHLz
liveK6dPs:livekq2IY
liveMLwoJ:liveUcZsS
liveawLoL:liveuNoce
liveBiXqb:liveVVVl7
liveqIV8z:liveRfvLA
liveiFkP1:liveiD8hw
liveMAA9l:livedCfIX
liveoUMSN:liveQkFdl
live4fwRy:livenu1tl
livewIsrT:liveUrjIt
livenqXfE:liveiNCVm
liveoiiSl:livemSkpP
livef3xCx:live9OxXe
live842lX:liveyfgt9
liveRZWx2:liveFdY1z
liveBCvy3:livez62ur
liveE5AaQ:liveJRkqM
liveMdFCj:live9Lz2r
live1R2sq:liveQce5E


In [11]:
conn = sqlite3.connect('participants.db')
cur = conn.cursor()

cur.execute('SELECT uniqueid, datastring FROM turkdemo')
for row in cur:
    try:
        container = json.loads(row[1])
        container = container['questiondata']
        if 'user_id' in container:
            id = container['user_id']

            pix2pix = cv2.imread('./results/pix2pix_blend/'+id+'.png', 1)
            ours = cv2.imread('./results/blend/'+id+'.png', 1)

            result = np.concatenate([pix2pix, ours], axis=1)
            cv2.imwrite('./results/comparison/'+id+'.png', result)
            print(id)
    except:
        pass

livelGKhh:livecAe8A
liveqiCpO:livekxnSI
liveZalZG:live3mmqv
liverKKXQ:liveBmTbT
liveRrGr5:liveNQ5pJ
liveTnIGw:liveDDuPv
liveWCitR:liveT90xQ
livetUVW6:live3JoJ8
liveirB7B:liveALieQ
liveMQzbL:live86ZrM
livehnnVT:live8nbIU
liveUwrB8:liveNZ5pU
livecGEHE:liveSkNnu
liveu8J6I:livexahWM
livegIjan:liveq34iy
liveU6dmX:livebGMPY
liveiVMty:livee906x
live1odip:live4dECl
liveaXNpX:liveetqUy
liveVe3AD:liveDe6mQ
live4o2ot:live1VkMU
live70nqI:livezjkam
live5chJb:livekwxeg
livesacqn:liveKVY8k
liveljo8R:livevgXNH
live2ZWHQ:live1Vwq7
liveFOhlh:liveXGHLz
liveK6dPs:livekq2IY
liveMLwoJ:liveUcZsS
liveawLoL:liveuNoce
liveBiXqb:liveVVVl7
liveqIV8z:liveRfvLA
liveiFkP1:liveiD8hw
liveMAA9l:livedCfIX
liveoUMSN:liveQkFdl
live4fwRy:livenu1tl
livewIsrT:liveUrjIt
livenqXfE:liveiNCVm
liveoiiSl:livemSkpP
livef3xCx:live9OxXe
live842lX:liveyfgt9
liveRZWx2:liveFdY1z
liveBCvy3:livez62ur
liveE5AaQ:liveJRkqM
liveMdFCj:live9Lz2r
live1R2sq:liveQce5E


In [8]:
conn = sqlite3.connect('participants.db')
cur = conn.cursor()

cur.execute('SELECT uniqueid, datastring FROM turkdemo')
for row in cur:
    try:
        container = json.loads(row[1])
        container = container['questiondata']
        if 'user_id' in container:
            id = container['user_id']
            id = id.replace(':', '%')
            print(id)
    except:
        pass

livelGKhh%livecAe8A
liveqiCpO%livekxnSI
liveZalZG%live3mmqv
liverKKXQ%liveBmTbT
liveRrGr5%liveNQ5pJ
liveTnIGw%liveDDuPv
liveWCitR%liveT90xQ
livetUVW6%live3JoJ8
liveirB7B%liveALieQ
liveMQzbL%live86ZrM
livehnnVT%live8nbIU
liveUwrB8%liveNZ5pU
livecGEHE%liveSkNnu
liveu8J6I%livexahWM
livegIjan%liveq34iy
liveU6dmX%livebGMPY
liveiVMty%livee906x
live1odip%live4dECl
liveaXNpX%liveetqUy
liveVe3AD%liveDe6mQ
live4o2ot%live1VkMU
live70nqI%livezjkam
live5chJb%livekwxeg
livesacqn%liveKVY8k
liveljo8R%livevgXNH
live2ZWHQ%live1Vwq7
liveFOhlh%liveXGHLz
liveK6dPs%livekq2IY
liveMLwoJ%liveUcZsS
liveawLoL%liveuNoce
liveBiXqb%liveVVVl7
liveqIV8z%liveRfvLA
liveiFkP1%liveiD8hw
liveMAA9l%livedCfIX
liveoUMSN%liveQkFdl
live4fwRy%livenu1tl
livewIsrT%liveUrjIt
livenqXfE%liveiNCVm
liveoiiSl%livemSkpP
livef3xCx%live9OxXe
live842lX%liveyfgt9
liveRZWx2%liveFdY1z
liveBCvy3%livez62ur
liveE5AaQ%liveJRkqM
liveMdFCj%live9Lz2r
live1R2sq%liveQce5E
